<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/Tungsten_Execution_Engine_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=9ec72188b5516095ca7e130b574533d05ce7e42613c29fe4acc519ebcfe05be5
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


Tungsten Execution Engine занимается управлением памяти вне кучи (off-heap) для **уменьшения затрат на сборку мусора**. Делает **Whole-Stage Code Generation**: Автоматическую генерацию специализированного кода для выполнения этапов обработки данных. Проще говоря он **отвечает именно за ресурсы для физического выполнение вашего кода**, а все остальное делает Catalyst Optimizer. То есть и физический план составляется Catalyst - ом, но сами ресурсы выделяет и подбирает Tungsten.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("Tungsten Example") \
    .config("spark.sql.codegen.wholeStage", "true") \
    .getOrCreate()

data = [(i, f"Name_{i % 5}", i % 3, i % 2) for i in range(1, 1000001)]
df = spark.createDataFrame(data, ["id", "name", "mod3", "mod2"])

# Применяем фильтр и агрегацию, чтобы увидеть работу Tungsten
result = df.filter(col("mod2") == 1) \
           .groupBy("mod3") \
           .agg({"id": "sum"}) \
           .orderBy("mod3")

result.explain(True)

== Parsed Logical Plan ==
'Sort ['mod3 ASC NULLS FIRST], true
+- Aggregate [mod3#2L], [mod3#2L, sum(id#0L) AS sum(id)#13L]
   +- Filter (mod2#3L = cast(1 as bigint))
      +- LogicalRDD [id#0L, name#1, mod3#2L, mod2#3L], false

== Analyzed Logical Plan ==
mod3: bigint, sum(id): bigint
Sort [mod3#2L ASC NULLS FIRST], true
+- Aggregate [mod3#2L], [mod3#2L, sum(id#0L) AS sum(id)#13L]
   +- Filter (mod2#3L = cast(1 as bigint))
      +- LogicalRDD [id#0L, name#1, mod3#2L, mod2#3L], false

== Optimized Logical Plan ==
Sort [mod3#2L ASC NULLS FIRST], true
+- Aggregate [mod3#2L], [mod3#2L, sum(id#0L) AS sum(id)#13L]
   +- Project [id#0L, mod3#2L]
      +- Filter (isnotnull(mod2#3L) AND (mod2#3L = 1))
         +- LogicalRDD [id#0L, name#1, mod3#2L, mod2#3L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [mod3#2L ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(mod3#2L ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=26]
      +- HashAggregate(keys=[mod3#2L

Здесь опишем все построчно.

**Операция сортировки данных по столбцу mod3**. Tungsten оптимизирует сортировку, используя целостную генерацию кода, что уменьшает накладные расходы на выполнение сортировки.

**Операция обмена данных между разделами для выполнения распределенной сортировки и агрегации**. Tungsten оптимизирует операции обмена, эффективно управляя памятью и процессорными ресурсами. В данном случае это rangepartitioning, оптимизированное для сортировки данных по mod3.

**Partial_sum** указывает на промежуточную агрегацию, что также оптимизируется Tungsten.

**Фильтрация данных по условию** `isnotnull(mod2#3L) AND (mod2#3L = 1)`. Tungsten использует генерацию кода для выполнения фильтрации с высокой производительностью.

**Операция сканирования исходного RDD для извлечения данных.**

Также давайте дадим определение для **Whole-Stage Code Generation** (Целостная генерация кода) —это одна из ключевых оптимизаций Tungsten, которая позволяет объединить несколько операций в одну стадию(stage) и генерировать специализированный машинный код для их выполнения.

Хотя Tungsten Execution Engine не всегда явно упоминается в физическом плане выполнения, его оптимизации работают за кулисами, обеспечивая высокую производительность Spark.